# Compute Zonal Means


In [ ]:
%load_ext dotenv
%dotenv

import os
from importlib import reload

import pandas as pd

from cholera import ceda, lst, outbreaks, precip, sm, xarray_ as xr_

# Reload modules to avoid the need to restart the notebook kernel after making
# code changes within the modules.
reload(lst)
reload(precip)
reload(sm)
reload(xr_)
None

In [ ]:
def zonal_means_to_csv(zonal_means: pd.DataFrame) -> str:
    start_year, start_month, _ = zonal_means.index.min()
    end_year, end_month, _ = zonal_means.index.max()
    path = (
        f"../data/zonal-means-{zonal_means.columns[0]}-"
        f"{start_year}{start_month:02d}-{end_year}{end_month:02d}.csv"
    )
    zonal_means.to_csv(path)

    return path

In [ ]:
username = os.environ.get("CEDA_USERNAME")
password = os.environ.get("CEDA_PASSWORD")
ceda_cert_expiration = (
    ceda.auth(username=username, password=password) if username and password else None
)

print(
    f"CEDA OPeNDAP certificate is valid through {ceda_cert_expiration.isoformat()} UTC"
    if ceda_cert_expiration
    else "Skipping CEDA auth check since a username and password were not configured"
)

In [ ]:
regions = outbreaks.regions()
start_months = outbreaks.start_month_range()

# Entire monthly range of outbreaks, with a 3-month prelude, and setting the
# end date to the end of the month.
all_months = slice(
    start_months[0] - pd.offsets.MonthBegin(3),
    start_months[-1] + pd.offsets.MonthEnd(),
)
print(all_months)

In [ ]:
sm_zonal_means = sm.zonal_means(regions, time=all_months)
zonal_means_to_csv(sm_zonal_means)
sm_zonal_means

In [ ]:
precip_zonal_means = precip.zonal_means(regions, time=all_months)
zonal_means_to_csv(precip_zonal_means)
precip_zonal_means

In [ ]:
names = precip_zonal_means.index.names
precip_values = precip_zonal_means.iloc[:, 0].values
print(precip_values)
precip_zonal_means.iloc[:, 0]

In [ ]:
precip_m_1 = precip_zonal_means.shift(1)
precip_m_2 = precip_zonal_means.shift(2)
precip_m_3 = precip_zonal_means.shift(3)

print(precip_m_1.values)
print(precip_m_2.values)
print(precip_m_3.values)

In [ ]:
if ceda_cert_expiration:
    lst_zonal_means = lst.zonal_means(regions, time=all_months)
    zonal_means_to_csv(lst_zonal_means)
    display(lst_zonal_means)